# BASIC IMPORTS

In [73]:
from __future__ import annotations

import json
import typing
from datetime import datetime

import faker
import pydantic as pt
import pytz
from devtools import debug
from rich.console import Console
from rich.pretty import pprint

console = Console()

### Settings Parsing from ENV File

In [74]:
class _BaseConfig(pt.BaseSettings):
    fb: pt.AnyHttpUrl = pt.Field(default=...)
    passwd: pt.StrictStr = pt.Field(default=..., min_length=6, max_length=18)

    class Config(pt.BaseSettings.Config):
        env_file = ".test.env"
        allow_mutation = False


configs = _BaseConfig()

### Import The Json 

In [75]:
contents: typing.Mapping[str, typing.Any] = {}

with open("data.json", "r") as f:
    contents = json.load(f.buffer)

### A Typical Model Schema of pydantic

In [91]:
title_str = pt.constr(
    strict=True,
    min_length=5,
    max_length=18,
    strip_whitespace=True,
    regex=r"^([a-zA-Z]+( [a-zA-Z]+)+)$",
)
category = pt.constr(strict=True, to_lower=True)


class Product(pt.BaseModel):
    id: int
    title: title_str
    description: str = pt.Field(default=None, max_length=255)
    price: float = pt.Field(default=..., decimalPlaces=2)
    discount_percentage: float = pt.Field(default=..., alias="discountPercentage")
    rating: int = pt.Field(ge=1, le=10)
    stock: int
    brand: str
    category: category
    thumbnail: pt.AnyHttpUrl
    images: typing.Tuple[pt.AnyHttpUrl, ...]

    @pt.validator("images", pre=True)
    @classmethod
    def _handle_images(cls, value: typing.Tuple[str]) -> typing.Tuple[pt.AnyHttpUrl]:
        return tuple(pt.AnyHttpUrl(url=_link, scheme="https") for _link in value)

    def __eq__(self, other_model: Product) -> boolean:
        if not isinstance(other_model, Product):
            return False
        return other_model.id == self.id

    class Config:
        orm_mode = True
        allow_population_by_field_name = True

In [92]:
try:
    product_1 = Product(**contents)
    product_2 = Product(
        **{
            "id": 12,
            "title": "Brown Person",
            "description": "Royal_Mirage Sport Brown Perfu...",
            "price": 40.0,
            "discountPercentage": 15.66,
            "rating": 4,
            "stock": 52,
            "brand": "Royal_Mirage",
            "category": "fragrances",
            "thumbnail": "https://i.dummyjson.com/data/products/12/thumbnail.jpg",
            "images": [
                "https://i.dummyjson.com/data/products/12/1.jpg",
                "https://i.dummyjson.com/data/products/12/2.jpg",
                "https://i.dummyjson.com/data/products/12/3.png",
                "https://i.dummyjson.com/data/products/12/4.jpg",
                "https://i.dummyjson.com/data/products/12/thumbnail.jpg",
            ],
        }
    )
except pt.ValidationError as e:
    console.print_exception(show_locals=True)
else:
    print(product_1 == product_2)
    console.print_json(product_1.json(by_alias=True))
    console.print_json(product_2.json(by_alias=True))

True

{
  "id": 12,
  "title": "Brown Person",
  "description": "Royal_Mirage Sport Brown Perfu...",
  "price": 40.0,
  "discountPercentage": 15.66,
  "rating": 4,
  "stock": 52,
  "brand": "Royal_Mirage",
  "category": "fragrances",
  "thumbnail": "https://i.dummyjson.com/data/products/12/thumbnail.jpg",
  "images": [
    "https://i.dummyjson.com/data/products/12/1.jpg",
    "https://i.dummyjson.com/data/products/12/2.jpg",
    "https://i.dummyjson.com/data/products/12/3.png",
    "https://i.dummyjson.com/data/products/12/4.jpg",
    "https://i.dummyjson.com/data/products/12/thumbnail.jpg"
  ]
}

{
  "id": 12,
  "title": "Brown Person",
  "description": "Royal_Mirage Sport Brown Perfu...",
  "price": 40.0,
  "discountPercentage": 15.66,
  "rating": 4,
  "stock": 52,
  "brand": "Royal_Mirage",
  "category": "fragrances",
  "thumbnail": "https://i.dummyjson.com/data/products/12/thumbnail.jpg",
  "images": [
    "https://i.dummyjson.com/data/products/12/1.jpg",
    "https://i.dummyjson.com/data/products/12/2.jpg",
    "https://i.dummyjson.com/data/products/12/3.png",
    "https://i.dummyjson.com/data/products/12/4.jpg",
    "https://i.dummyjson.com/data/products/12/thumbnail.jpg"
  ]
}

### Enum With Pydantic Schema

In [72]:
import enum

from rich import print


class EnumStatus(str, enum.Enum):
    PENDING = "pending"
    SUCCESS = "success"
    DRAFT = "draft"


class EnumActiveStage(int, enum.Enum):
    ACTIVE = 1
    INACTIVE = 0


class Order(pt.BaseModel):
    id: int
    status: EnumStatus
    is_active: EnumActiveStage


order_1 = Order(id=1, status="pending", is_active=1)
order_2 = Order(id=1, status="success", is_active=0)
order_3 = Order(id=1, status="draft", is_active=1)

print([order_1, order_2, order_3])

[
    Order(id=1, status=<EnumStatus.PENDING: 'pending'>, is_active=<EnumActiveStage.ACTIVE: 1>),
    Order(id=1, status=<EnumStatus.SUCCESS: 'success'>, is_active=<EnumActiveStage.INACTIVE: 0>),
    Order(id=1, status=<EnumStatus.DRAFT: 'draft'>, is_active=<EnumActiveStage.ACTIVE: 1>)
]

#### ACCESS ENUM VALUE 

In [68]:
print(order_1.is_active == EnumActiveStage.ACTIVE)
print(order_1.status == EnumStatus.PENDING)
print()
print(f"Order Status Name: {order_1.status.name}")
print(f"Order Status Value: {order_1.status.value}")
print()
print(f"Order Active Name: {order_1.is_active.name}")
print(f"Order Active Value: {order_1.is_active.value}")
print()
print(order_1.dict())

True

True

Order Status Name: PENDING

Order Status Value: pending

Order Active Name: ACTIVE

Order Active Value: 1

{'id': 1, 'status': <EnumStatus.PENDING: 'pending'>, 'is_active': <EnumActiveStage.ACTIVE: 1>}